### dev

In [1]:
import os
import numpy as np
import pandas as pd
import zarr
import glob
import btrack
import cv2
from PIL import Image, ImageDraw, ImageFont
from tqdm.auto import tqdm

In [3]:
# extract single cell df
sc_df_fn = '/mnt/SYNO/macrohet_syno/results/dfs/arx/dt_df.pkl' # time intensive step
df = pd.read_pickle(sc_df_fn)


In [4]:
# Prepare font and color settings
font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"  # Update this path to your font file
font = ImageFont.truetype(font_path, 42)
text_color = (0, 255, 255)  # Yellow color in RGB format
# Define the side length
side_length = 750
# and other image properties
track_scale_factor = 5.04
image_scale_m_per_pixel = 1.4949402023919043e-07
image_scale_um_per_pixel = image_scale_m_per_pixel*1E6
image_width = image_height = 6048
last_seg_fn = []

# Bulk generation

In [5]:
for unique_ID in tqdm(df['ID'].unique(), total = len(df['ID'].unique())):
    
    try: 
        if 'ND0003' not in unique_ID:
            continue
        # if os.path.exists(f"/mnt/SYNO/macrohet_syno/glimpses/{unique_ID}.mp4"):
        #     print(f'Path exists {unique_ID}')
        #     continue 
            
        sc_df = df[df['ID'] == unique_ID]
        
        # get the acq id information
        acq_ID = row, column = int(unique_ID.split('.')[1]), int(unique_ID.split('.')[2]) 
        cell_ID = int(unique_ID.split('.')[0])
        expt_ID = unique_ID.split('.')[-1]
        
        # assign the proper channel enumeration
        mphi_channel = 0 if expt_ID == 'PS0000' else 1
        mtb_channel = 1 if expt_ID == 'PS0000' else 0
        
        # if no side length provided then estimate based on max mphi area
        if not side_length:
            # Calculate the side length for cropping based on the square root of the area
            side_length = int(np.sqrt(sc_df['Mphi Area (µm)'].max())) * 2
        
        # preload the images
        image_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/zarr/{acq_ID}.zarr'
        zarr_group = zarr.open(image_dir, mode='r')
        images = zarr_group.images
        sample_image = images[0,0,0,...]
        
        
        # Load segmentation if necessary
        seg_fn = glob.glob(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/*/{acq_ID}.h5')[0]
        if seg_fn != last_seg_fn:
            with btrack.io.HDF5FileHandler(seg_fn, 'r', obj_type='obj_type_1') as reader:
                segmentation = reader.segmentation
            last_seg_fn = seg_fn
        
        # load the segmentation 
        rgb_stack = []
        
        # iterate over each frame/data point
        for i, t in tqdm(enumerate(sc_df['Time (hours)']), total = len(sc_df), leave = False, desc = f'Creating glimpse {unique_ID}'):
            
            sc_df_t = sc_df[sc_df['Time (hours)'] == t]
            # Extract xy coordinates and transpose for python and area from the cell information
            y_coord, x_coord, area, t, f, mtb = sc_df_t.loc[:, ['x', 'y', 'Mphi Area (µm)', 'Time (hours)', 'Frame', 'Mtb Area (µm)']].values[0]
        
            # Scale according to tracking shrinkage
            y_coord, x_coord = y_coord * track_scale_factor, x_coord * track_scale_factor
        
            # Calculate the cropping boundaries
            x_start = int(x_coord - side_length / 2)
            x_end = int(x_coord + side_length / 2)
            y_start = int(y_coord - side_length / 2)
            y_end = int(y_coord + side_length / 2)
            
            # Pad the boundaries if they exceed the image dimensions
            if x_start < 0:
                x_pad = abs(x_start)
                x_start = 0
            else:
                x_pad = 0
        
            if x_end > sample_image.shape[0]:
                x_pad_end = x_end - sample_image.shape[0]
                x_end = sample_image.shape[0]
            else:
                x_pad_end = 0
        
            if y_start < 0:
                y_pad = abs(y_start)
                y_start = 0
            else:
                y_pad = 0
        
            if y_end > sample_image.shape[1]:
                y_pad_end = y_end - sample_image.shape[1]
                y_end = sample_image.shape[1]
            else:
                y_pad_end = 0
        
            # Crop the image
            cropped_image = images[int(f), :, 0, x_start:x_end, y_start:y_end]
        
            # Pad the cropped image if necessary
            cropped_image = np.pad(cropped_image, ((0, 0), (x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
            
            # extract the gfp and rfp channels to apply some vis techn
            gfp = cropped_image[mphi_channel, ...]
            rfp = cropped_image[mtb_channel, ...]
            
            # clip the images so that the contrast is more apparent
            contrast_lim_gfp = np.clip(gfp, 358, 5886)
            contrast_lim_rfp = np.clip(rfp, 480, 1300)
            
            norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
            norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
            
            # Create an empty RGB image with the same shape as the input image
            rgb_image = np.zeros((contrast_lim_gfp.shape[0], contrast_lim_gfp.shape[1], 3), dtype=np.uint16)
            
            # Assign the first channel to the green channel of the RGB image
            rgb_image[..., 1] = norm_gfp
            
            # Assign the second channel to the red and blue channels of the RGB image to create magenta
            rgb_image[..., 0] = norm_rfp
            rgb_image[..., 2] = norm_rfp
            
            # scale down to 8bit
            rgb_image = np.uint8(rgb_image >> 8)
        
            
            # load mask (singular)
            cropped_masks = segmentation[int(f), x_start:x_end, y_start:y_end]
            
            # Pad the cropped image if necessary
            cropped_masks = np.pad(cropped_masks, ((x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
        
            # extract only that segment
            seg_ID = cropped_masks[int(cropped_masks.shape[0] / 2), int(cropped_masks.shape[1] / 2)]
           
            if seg_ID == 0:
                instance_mask = np.zeros((side_length, side_length), dtype = np.uint8)
            else:
                instance_mask = (cropped_masks == seg_ID).astype(np.uint8)
        
            # draw outline
            contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(rgb_image, contours, -1, (0, 2 ** 8, 2 ** 8), thickness=2)  # make 8bit
            
            # Convert to PIL image for text overlay
            pil_image = Image.fromarray(rgb_image)
            draw = ImageDraw.Draw(pil_image)
        
            # Bottom left corner text
            bottom_left_text = f"{t} hours"
            draw.text((10, rgb_image.shape[0] - 80), bottom_left_text, font=font, fill=text_color)
        
            # Bottom right corner text
            bottom_right_text = '20µm'
            text_size = font.getbbox(bottom_right_text)
            text_width = text_size[2] - text_size[0]  # Calculate text width
            bottom_right_text_position = (rgb_image.shape[1] - text_width - 40, rgb_image.shape[0] - 80)
            draw.text(bottom_right_text_position, bottom_right_text, font=font, fill=text_color)
        
            # Calculate line length in pixels for a 20 micrometer scale bar
            line_length_pixels = int(20 / image_scale_um_per_pixel)
            line_start = (bottom_right_text_position[0], bottom_right_text_position[1] + 55)
            line_end = (line_start[0] + line_length_pixels, line_start[1])
            draw.line([line_start, line_end], fill=text_color, width=10)
        
            # Top left corner text
            draw.text((10, 10), unique_ID, font=font, fill=text_color)
        
            # Top right corner text
            mtb_value = f"Mtb:{mtb:.2f}µm²"
            text_size = font.getbbox(mtb_value)
            text_width = text_size[2] - text_size[0]  # Calculate text width
            top_right_text_position = (rgb_image.shape[1] - text_width - 10, 10)
            draw.text(top_right_text_position, mtb_value, font=font, fill=text_color)
        
            # Convert back to OpenCV image
            rgb_image = np.array(pil_image)
            
            # Resize image to consistent shape
            rgb_image_resized = cv2.resize(rgb_image, (side_length, side_length), interpolation=cv2.INTER_AREA)

            # add to stack
            rgb_stack.append(rgb_image_resized)
    
        # compile into array
        rgb_stack = np.stack(rgb_stack, axis = 0)
        
        # Get the dimensions of the first frame
        height, width, _ = rgb_stack[0].shape
        
        # Define the frame rate (number of frames per second)
        frame_rate = len(rgb_stack) / 20  # Total frames divided by total seconds
        output_file = f"/mnt/SYNO/macrohet_syno/results/glimpse_dt_graphs/glimpses/{unique_ID}.mp4"
        # Initialize VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, frame_rate, (width, height))
        
        # Write each frame to the video file
        for frame in rgb_stack:
            out.write(frame)
        
        # Release the VideoWriter object
        out.release()
        
        print(f"Video {unique_ID} saved successfully.")

    except Exception as e:
        error_message = f"Error: {e}\nUnique ID: {unique_ID}\n"
        with open("error_log.txt", "a") as file:
            file.write(error_message)
        

  0%|          | 0/13405 [00:00<?, ?it/s]

[INFO][2024/09/09 05:00:03 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/cpv3/(3, 1).h5...
[INFO][2024/09/09 05:00:24 PM] Loading segmentation (154, 6048, 6048)
[INFO][2024/09/09 05:00:24 PM] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/cpv3/(3, 1).h5


Creating glimpse 220.3.1.ND0003:   0%|          | 0/71 [00:00<?, ?it/s]

Video 220.3.1.ND0003 saved successfully.


Creating glimpse 207.3.1.ND0003:   0%|          | 0/76 [00:00<?, ?it/s]

...

Creating glimpse 1796.6.12.ND0003:   0%|          | 0/74 [00:00<?, ?it/s]

Creating glimpse 1803.6.12.ND0003:   0%|          | 0/82 [00:00<?, ?it/s]

Creating glimpse 1815.6.12.ND0003:   0%|          | 0/79 [00:00<?, ?it/s]

Creating glimpse 1822.6.12.ND0003:   0%|          | 0/74 [00:00<?, ?it/s]

Creating glimpse 1874.6.12.ND0003:   0%|          | 0/70 [00:00<?, ?it/s]

In [11]:
error_message

"Error: nothing found at path ''\nUnique ID: 2768.4.12.ND0003\n"

# Single instance

In [297]:
df['ID'].unique()

array(['426.3.4.PS0000', '427.3.4.PS0000', '435.3.4.PS0000', ...,
       '1815.6.12.ND0003', '1822.6.12.ND0003', '1874.6.12.ND0003'],
      dtype=object)

In [298]:
unique_ID = '426.3.4.PS0000' # '211.3.5.PS0000'

In [9]:
sc_df = df[df['ID'] == unique_ID]
        
# get the acq id information
acq_ID = row, column = int(unique_ID.split('.')[1]), int(unique_ID.split('.')[2]) 
cell_ID = int(unique_ID.split('.')[0])
expt_ID = unique_ID.split('.')[-1]

# assign the proper channel enumeration
mphi_channel = 0 if expt_ID == 'PS0000' else 1
mtb_channel = 1 if expt_ID == 'PS0000' else 0

# if no side length provided then estimate based on max mphi area
if not side_length:
    # Calculate the side length for cropping based on the square root of the area
    side_length = int(np.sqrt(sc_df['Mphi Area (µm)'].max())) * 2

# preload the images
image_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/zarr/{acq_ID}.zarr'
zarr_group = zarr.open(image_dir, mode='r')
images = zarr_group.images
sample_image = images[0,0,0,...]


# Load segmentation if necessary
seg_fn = glob.glob(f'/mnt/SYNO/macrohet_syno/{expt_ID}/labels/*/{acq_ID}.h5')[0]
if seg_fn != last_seg_fn:
    with btrack.io.HDF5FileHandler(seg_fn, 'r', obj_type='obj_type_1') as reader:
        segmentation = reader.segmentation
    last_seg_fn = seg_fn

# load the segmentation 
rgb_stack = []

# iterate over each frame/data point
for i, t in tqdm(enumerate(sc_df['Time (hours)']), total = len(sc_df), leave = False, desc = f'Creating glimpse {unique_ID}'):
    
    sc_df_t = sc_df[sc_df['Time (hours)'] == t]
    # Extract xy coordinates and transpose for python and area from the cell information
    y_coord, x_coord, area, t, f, mtb = sc_df_t.loc[:, ['x', 'y', 'Mphi Area (µm)', 'Time (hours)', 'Frame', 'Mtb Area (µm)']].values[0]

    # Scale according to tracking shrinkage
    y_coord, x_coord = y_coord * track_scale_factor, x_coord * track_scale_factor

    # Calculate the cropping boundaries
    x_start = int(x_coord - side_length / 2)
    x_end = int(x_coord + side_length / 2)
    y_start = int(y_coord - side_length / 2)
    y_end = int(y_coord + side_length / 2)
    
    # Pad the boundaries if they exceed the image dimensions
    if x_start < 0:
        x_pad = abs(x_start)
        x_start = 0
    else:
        x_pad = 0

    if x_end > sample_image.shape[0]:
        x_pad_end = x_end - sample_image.shape[0]
        x_end = sample_image.shape[0]
    else:
        x_pad_end = 0

    if y_start < 0:
        y_pad = abs(y_start)
        y_start = 0
    else:
        y_pad = 0

    if y_end > sample_image.shape[1]:
        y_pad_end = y_end - sample_image.shape[1]
        y_end = sample_image.shape[1]
    else:
        y_pad_end = 0

    # Crop the image
    cropped_image = images[int(f), :, 0, x_start:x_end, y_start:y_end]

    # Pad the cropped image if necessary
    cropped_image = np.pad(cropped_image, ((0, 0), (x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
    
    # extract the gfp and rfp channels to apply some vis techn
    gfp = cropped_image[mphi_channel, ...]
    rfp = cropped_image[mtb_channel, ...]
    
    # clip the images so that the contrast is more apparent
    contrast_lim_gfp = np.clip(gfp, 358, 5886)
    contrast_lim_rfp = np.clip(rfp, 480, 1300)
    
    norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    
    # Create an empty RGB image with the same shape as the input image
    rgb_image = np.zeros((contrast_lim_gfp.shape[0], contrast_lim_gfp.shape[1], 3), dtype=np.uint16)
    
    # Assign the first channel to the green channel of the RGB image
    rgb_image[..., 1] = norm_gfp
    
    # Assign the second channel to the red and blue channels of the RGB image to create magenta
    rgb_image[..., 0] = norm_rfp
    rgb_image[..., 2] = norm_rfp
    
    # scale down to 8bit
    rgb_image = np.uint8(rgb_image >> 8)

    
    # load mask (singular)
    cropped_masks = segmentation[int(f), x_start:x_end, y_start:y_end]
    
    # Pad the cropped image if necessary
    cropped_masks = np.pad(cropped_masks, ((x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')

    # extract only that segment
    seg_ID = cropped_masks[int(cropped_masks.shape[0] / 2), int(cropped_masks.shape[1] / 2)]
   
    if seg_ID == 0:
        instance_mask = np.zeros((side_length, side_length), dtype = np.uint8)
    else:
        instance_mask = (cropped_masks == seg_ID).astype(np.uint8)

    # draw outline
    contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(rgb_image, contours, -1, (0, 2 ** 8, 2 ** 8), thickness=2)  # make 8bit
    
    # Convert to PIL image for text overlay
    pil_image = Image.fromarray(rgb_image)
    draw = ImageDraw.Draw(pil_image)

    # Bottom left corner text
    bottom_left_text = f"{t} hours"
    draw.text((10, rgb_image.shape[0] - 80), bottom_left_text, font=font, fill=text_color)

    # Bottom right corner text
    bottom_right_text = '20µm'
    text_size = font.getbbox(bottom_right_text)
    text_width = text_size[2] - text_size[0]  # Calculate text width
    bottom_right_text_position = (rgb_image.shape[1] - text_width - 40, rgb_image.shape[0] - 80)
    draw.text(bottom_right_text_position, bottom_right_text, font=font, fill=text_color)

    # Calculate line length in pixels for a 20 micrometer scale bar
    line_length_pixels = int(20 / image_scale_um_per_pixel)
    line_start = (bottom_right_text_position[0], bottom_right_text_position[1] + 55)
    line_end = (line_start[0] + line_length_pixels, line_start[1])
    draw.line([line_start, line_end], fill=text_color, width=10)

    # Top left corner text
    draw.text((10, 10), unique_ID, font=font, fill=text_color)

    # Top right corner text
    mtb_value = f"Mtb:{mtb:.2f}µm²"
    text_size = font.getbbox(mtb_value)
    text_width = text_size[2] - text_size[0]  # Calculate text width
    top_right_text_position = (rgb_image.shape[1] - text_width - 10, 10)
    draw.text(top_right_text_position, mtb_value, font=font, fill=text_color)

    # Convert back to OpenCV image
    rgb_image = np.array(pil_image)
    
    # Resize image to consistent shape
    rgb_image_resized = cv2.resize(rgb_image, (side_length, side_length), interpolation=cv2.INTER_AREA)

    # add to stack
    rgb_stack.append(rgb_image_resized)

# compile into array
rgb_stack = np.stack(rgb_stack, axis = 0)

# Get the dimensions of the first frame
height, width, _ = rgb_stack[0].shape

# Define the frame rate (number of frames per second)
frame_rate = len(rgb_stack) / 20  # Total frames divided by total seconds
output_file = f"/mnt/SYNO/macrohet_syno/glimpses/{unique_ID}.mp4"
# Initialize VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, frame_rate, (width, height))

# Write each frame to the video file
for frame in rgb_stack:
    out.write(frame)

# Release the VideoWriter object
out.release()

print(f"Video {unique_ID} saved successfully.")

NameError: name 'last_seg_fn' is not defined